In [1]:
# Calculate water mass formation for a specific density value from NEMO data

In [39]:
import numpy as np
import pandas as pd
import xarray as xr
import xarray.ufuncs as xu
import scipy.io as io
import gsw
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
# SPECIFY DATASET LOCATION
rootdir = '/home/ocean1/DRAKKAR/ORCA025.L75-GJM189-S/'
r = 27
dr = 0.1
# NA region
xv = np.arange(700,1300,1)
yv = np.arange(500,1021,1)

In [41]:
# LOAD DATA
# Load data files
t = '*'
ds = xr.open_mfdataset(rootdir+'2012/ORCA025.L75-GJM189_'+t+'_gridT.nc',chunks={'x':1442,'y':1021})
# Load grid data
grd = xr.open_dataset(rootdir+'GRID/ORCA025.L75-GJM189_mesh_hgr.nc')
# Load mask
# Prepared in Matlab before.
init_mask = io.loadmat('/home/ocean2/graemem/dwfproxy/DRAKKAR/matfiles/init_mask.mat')
mask = xr.DataArray(np.array(init_mask['mask']),coords=[grd.x,grd.y])

In [46]:
M=calc_wmf(ds.isel(x=xv,y=yv),grd.isel(x=xv,y=yv),r,dr)
print M

<xarray.DataArray (time_counter: 73)>
dask.array<div-4f7..., shape=(73,), dtype=float64, chunksize=(1,)>
Coordinates:
    deptht        float32 0.50576
  * time_counter  (time_counter) datetime64[ns] 2011-12-31T12:00:00 ...


In [36]:
def calc_buoyancyflux(ds):
    import xarray.ufuncs as xu
    import gsw
    
    # Calculate buoyancy flux
    r0 = gsw.rho_xarray(ds.vosaline.isel(deptht=0),ds.votemper.isel(deptht=0),0)
    alpha = gsw.alpha_xarray(ds.vosaline.isel(deptht=0),ds.votemper.isel(deptht=0),0)
    beta = gsw.beta_xarray(ds.vosaline.isel(deptht=0),ds.votemper.isel(deptht=0),0)
    D_T = -(alpha/gsw.cp0)*ds.sohefldo
    D_S = r0*beta*ds.vosaline.isel(deptht=0)*ds.sowaflup/1000
    
    return D_T + D_S

def calc_wmt(ds,grd,r,dr):
    import numpy as np
    import xarray.ufuncs as xu
    import gsw
    
    # Calculate sea-surface density
    r0 = gsw.rho_xarray(ds.vosaline.isel(deptht=0),ds.votemper.isel(deptht=0),0)
    # Calculate buoyancy flux
    D_in = calc_buoyancyflux(ds)
    
    # Multiply by area of grid cell
    D_in_area = D_in*grd.e1t*grd.e2t
    
    # Calculate summed buoyancy flux for surface densities greater than r
    # Generate new DataArray for values above and below the target density
    Dbelow = D_in_area.where(r0>r-dr).sum(['x','y','t'])
    Dtarget = D_in_area.where(r0>r).sum(['x','y','t'])
    Dabove = D_in_area.where(r0>r+dr).sum(['x','y','t'])
    # Dimension 't' is bug due to naming of dimensions in grid file
    # Perform differentiation on these DataArrays
    
    # TRANSFORMATION
    # First order central difference of D wrt rho
    return -(1e-6)*(Dabove-Dbelow)/(2*dr)

def calc_wmf(ds,grd,r,dr):
    import numpy as np
    import xarray.ufuncs as xu
    import gsw
    
    # Calculate sea-surface density
    r0 = gsw.rho_xarray(ds.vosaline.isel(deptht=0),ds.votemper.isel(deptht=0),0)
    # Calculate buoyancy flux
    D_in = calc_buoyancyflux(ds)
    
    # Multiply by area of grid cell
    D_in_area = D_in*grd.e1t*grd.e2t
    
    # Calculate summed buoyancy flux for surface density greater than r
    # Generate new DataArray for values above and below the target density
    Dbelow = D_in_area.where(r0>r-dr).sum(['x','y','t'])
    Dtarget = D_in_area.where(r0>r).sum(['x','y','t'])
    Dabove = D_in_area.where(r0>r+dr).sum(['x','y','t'])
    # Dimension 't' is bug due to naming of dimensions in grid file
    # Perform differentiation on these DataArrays
    
    # FORMATION
    # Second order central differences of D wrt rho
    return (1e-6)*(Dabove-2*Dtarget+Dbelow)/(dr**2);